<a href="https://colab.research.google.com/github/OrensteinLab/PrimerDesigner/blob/main/Tracking_Timings_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pre-Calculations

##Installs & Imports

In [ ]:
!pip install -U bokeh seaborn pandas
!pip install git+https://github.com/FordyceLab/seequence.git#egg=seequence
!pip install primer3-py biopython pandarallel
!pip install pulp
!pip install gurobipy
!pip install biopython==1.75




  Cloning https://github.com/FordyceLab/seequence.git to /tmp/pip-install-kv3fra97/seequence_8828efea384444d2840e97ba922276b7
  Running command git clone --filter=blob:none --quiet https://github.com/FordyceLab/seequence.git /tmp/pip-install-kv3fra97/seequence_8828efea384444d2840e97ba922276b7
  Resolved https://github.com/FordyceLab/seequence.git to commit 3ea730537fcf5b7ef807ebf6e057f5bf4e875bb9
  Preparing metadata (setup.py) ... done


In [ ]:
# IMPORTS
import time

import random as rand
from itertools import product

from seequence.view import *
from seequence.color import *

from pandarallel import pandarallel as pl
pl.initialize()

import primer3 as p3
from Bio.Seq import Seq
from Bio.SeqUtils import GC, seq1, seq3
from Bio.SeqUtils.CodonUsage import SharpEcoliIndex, SynonymousCodons

import itertools as it
import numpy as np
import pandas as pd
pd.set_option('display.precision', 1)
import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.models.annotations import Span
from bokeh.models import Select
from bokeh.layouts import column
from IPython.display import display, clear_output

import gurobipy as gp

import pickle

import tracemalloc
import matplotlib.pyplot as plt

def revcomp(seq):
  return str(Seq(seq).reverse_complement())
def translate(seq):
  return str(Seq(seq).translate())

# clear_output()
print('Ready')

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Ready


In [ ]:
def calcOffTarget(primer, seq, start):
  fl,fr = seq[:primer.start-start], seq[primer.stop-start:]
  rl,rr = revcomp(fl), revcomp(fr)

  res_fl = pcr.calcHeterodimer(primer.seq, fl).todict()
  res_fr = pcr.calcHeterodimer(primer.seq, fr).todict()
  res_rl = pcr.calcHeterodimer(primer.seq, rl).todict()
  res_rr = pcr.calcHeterodimer(primer.seq, rr).todict()

  ot_tm = max(res_fl['tm'], res_fr['tm'], res_rl['tm'], res_rr['tm'])
  # ot_dg = min(res_fl['dg'], res_fr['dg'], res_rl['dg'], res_rr['dg'])*1e-3
  return ot_tm

def n_subsequences(sequence, lmin, lmax):
  print(sum(len(sequence) - l + 1 for l in range(lmin, lmax+1)))

def subsequences(sequence, lmin, lmax): #Generates all subsequences w/ all poss. start-stop pairs
  ls = []
  for j in range(lmin, lmax+1): #length
    for i in range(len(sequence)-j+1): #starting index
      start = i
      stop = i+j
      ls.append([sequence[start:stop], start, stop, stop-start])
  return pd.DataFrame(ls, columns=['seq','start','stop','len'])

## Primer Data
##### Creates the primer pandas dataframe containing information about every possible primer in the protein coding sequence. The function calculates various primer parameters including gc content, delta G and melting temepratures. The cost is also calculated for every primer

In [ ]:
def create_primer_df(sequence_nt):
  # convention: start index of r-primers will be 3' (i.e. start < stop)
  primer_f = pd.DataFrame(columns=['seq','start','stop','fr','len'])
  primer_f[['seq','start','stop','len']] = subsequences(sequence_nt, primer_lmin, primer_lmax)
  primer_f['fr'] = 'f'

  #Shifting so that 0 is at the start of mutreg (upstream has negative values)
  primer_f['start'] = primer_f.start - len(upstream_nt)
  primer_f['stop'] = primer_f.stop - len(upstream_nt)

  #Creating reverse primers at same locations
  primer_r = primer_f[['seq','start','stop','fr','len']].copy()
  primer_r['fr'] = 'r'
  primer_r['seq'] = primer_r.seq.apply(revcomp)

  #Concatenating Forward & Reverse
  primer_df = pd.concat([primer_f,primer_r])
  primer_df.sort_values(by=['start','stop','fr'], inplace=True)

  #Calculating "Cost" Values
  primer_df['gc'] = primer_df.seq.apply(GC)
  primer_df['tm'] = primer_df.seq.apply(pcr.calcTm)
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
  primer_df['hp_tm'] = res.apply(lambda res: res['tm'])
  primer_df['hp_dg'] = res.apply(lambda res: res['dg']*1e-3)
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHomodimer(s).todict())
  primer_df['ho_tm'] = res.apply(lambda res: res['tm'])
  primer_df['ho_dg'] = res.apply(lambda res: res['dg']*1e-3)

  def primer_cost(primer):
    # calculates primer cost based on homodimer an haipin delta G, tm cost, and len cost
    hp_dg_max = -4
    ho_dg_max = -8
    tm_min = 58

    tm_cost = max(0, tm_min-primer.tm)**1.7
    hp_cost = max(0, hp_dg_max - primer.hp_dg)**1.2
    ho_cost = max(0, ho_dg_max - primer.ho_dg)**1.2
    len_cost = primer.len*1e-5

    cost = hp_cost + ho_cost + len_cost + tm_cost
    return cost

  primer_df['cost'] = primer_df.parallel_apply(primer_cost, axis=1)
  primer_df['log10cost'] = primer_df.cost.apply(np.log10)

  primer_df.reset_index(inplace=True)
  primer_f = primer_df.query('fr=="f"').reset_index(drop=True)
  primer_r = primer_df.query('fr=="r"').reset_index(drop=True)
  primer_df.set_index(['start','stop','fr'], inplace=True)

  return primer_f, primer_df

def check_threshold(tm,gc):
  # returns false only if the threshold flag is on and primers did not pass threshold
  if not apply_threshold:
    return True
  else:
    if min_gc <= gc <= max_gc and  min_tm <= tm <=max_tm:
      return True
    else:
      return False



## Primer Class
##### Creates a primer class which is used for the creation of the primer graph

In [ ]:
class Primer:
  def __init__(self,primer_df,start,stop,is_r=False):
    assert start < stop
    self.start = start
    self.stop = stop
    self.is_r = is_r #forward or reverse
    self.l = stop-start #length
    self.primer_df = primer_df
    self.w = primer_df.at[self.tup(),'cost'] #total cost value; the fancy notation is b/c
                                             #of the hierarchal lookup system in panda.df

  def __str__(self):
    return ' '.join(map(str,(self.start, self.stop, self.is_r)))
  def __repr__(self):
    return f'{("r" if self.is_r else"f")}({self.start},{self.stop})'
  def tup(self):
    return (self.start,self.stop,("r" if self.is_r else"f"))


def actions(primer_df,primer): #returning possible counterparts (forward -> reverse; reverse -> forward)
                    #i.e. this method gets the "neighbors"
  if not primer.is_r:  # fwd
    for oligo_l, primer_l in it.product(reversed(range(oligo_lmin, oligo_lmax+1)),
                                        range(primer_lmin,primer_lmax+1)):

      stop = primer.start + oligo_l
      start = stop - primer_l

      if apply_threshold:
        # finds tm of forward and reverse primers
        tm_f=primer_df.at[primer.tup(),'tm']
        tm_r=primer_df.at[(start,stop,"r"),'tm']

        # if tm difference is larger then max_difference threshold do not add primer to graph
        if abs(tm_f-tm_r)>max_difference:
          continue

      yield Primer(primer_df,start, stop, is_r=True)

  elif primer.is_r:  # rev
    for overlap_l, primer_l in it.product(reversed(range(overlap_lmin, overlap_lmax+1)),
                                          range(primer_lmin,primer_lmax+1)):

      start = primer.stop - overlap_l
      stop = start + primer_l


      # filter
      no_split = (primer.start - stop) >= primer.start%3
      if (stop > primer.start) or (not no_split): ## redundant to check first condition?
        continue
      yield Primer(primer_df,start, stop)

## Graph DFS
##### Recursive function used for creating the primer graph representing all possible forward and reverse primer combinations

In [ ]:
def dfs(G, primer, primer_df, mutreg_l): #CREATING the graph

  tm=primer_df.at[primer.tup(),'tm']
  gc=primer_df.at[primer.tup(),'gc']

  if (primer.start >= mutreg_l) and primer.is_r and check_threshold(tm,gc) :  # base case (end)
    G.add_edge(primer.tup(),'d', weight=0.) #G is global variable defined in next section
    return

  for next_primer in actions(primer_df,primer):

    is_new = not G.has_node(next_primer.tup()) # check if primer node exists already

    tm=primer_df.at[next_primer.tup(),'tm']
    gc=primer_df.at[next_primer.tup(),'gc']

    passes_threshold= check_threshold(tm,gc) # check if primer passes threshold

    # only add edge if primer passes threshold
    if passes_threshold:
      G.add_edge(primer.tup(),next_primer.tup(), weight=next_primer.w) #weight is the cost of the new primer
    if passes_threshold and is_new:
      dfs(G,next_primer,primer_df,mutreg_l)

def paths_ct(G, u, d): #total # of paths between two points
    if u == d:
        return 1
    else:
        if not G.nodes[u]: #npaths attribute is the # of paths out of u
            G.nodes[u]['npaths'] = sum(paths_ct(G, c, d) for c in G.successors(u))
        return G.nodes[u]['npaths']

## Greedy Algorithms
##### Runs the baseline greedy algorithm which is compared to PrimerDesigner

In [ ]:
def run_greedy(graphs, primer_dfs,multiple_forbidden,protein_names):

  path_ls = []

  for protein in protein_names:
    nodes_to_remove = []
    G = graphs[protein]
    G_sub = G.copy()
    for i,path in enumerate(path_ls):
      other_protein = protein_names[i]
      for p,n in it.product(path,G_sub.nodes()):
        if n=='s' or n=='d':
          continue
        pn_forbidden = ((p[0],p[1]),(n[0],n[1])) in multiple_forbidden[(other_protein,protein)]
        if pn_forbidden:
          nodes_to_remove.append(n)
    G_sub.remove_nodes_from(set(nodes_to_remove))
    try:
      path_ls.append([primer for primer in nx.algorithms.shortest_path(G_sub,'s','d', weight='weight')][1:-1])
    except:
      print(f'WARNING: No feasible primer sequence for lib_{i}; reduce number of libraries or relax constraints.')

  # # https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
  primer_set = pd.DataFrame()
  for i,primer_ls in enumerate(path_ls):
    protein = protein_names[i]
    primer_df = primer_dfs[protein]
    primer_set1 = primer_df.loc[primer_ls].copy().reset_index()
    primer_set1['lib_i'] = protein
    primer_set = pd.concat([primer_set, primer_set1])
  primer_set['tile_i'] = primer_set.groupby(['lib_i','fr']).cumcount()
  primer_set = primer_set[['lib_i','tile_i']+primer_set.columns[:-2].to_list()]

  #print(primer_set.groupby('lib_i').cost.sum())
  cost = primer_set.cost.sum()

  return path_ls, cost


##ILP Functions

## Gurobi Setup
##### The user should input their Gurobi license details for the ILP part

In [ ]:
def get_model():
  params = {
  "WLSACCESSID":"",
  "WLSSECRET":"",
  "LICENSEID":
  }
  env = gp.Env(params=params)

  # Create the model within the Gurobi environment
  model = gp.Model('min-sum', env=env)
  return model

## Single protein constraints

In [ ]:

def single_pairs(protein_names,sequences_nt):

  single_forbidden={}
  for name,sequence in zip(protein_names,sequences_nt):
    # find forbidden_pairs in single protein
    pairs_finder = PairFinder(sequence)
    forbidden_pairs = pairs_finder.find_all_pairs()
    single_forbidden[name]=forbidden_pairs

  return single_forbidden

def single_constraints(graph,forbidden_pairs,model):

    #Getting Nodes/Edges
    graph_edges = graph.edges(data=True)
    graph_nodes = [node for node in graph.nodes if node != 's' and node != 'd'] #removing s & d nodes

    #Converting Graphs to Lists of model variables
    ij = gp.tuplelist()
    w_ij = gp.tupledict()

    for edge in graph_edges:
      l = (str(edge[0]), str(edge[1])) #i, j
      ij.append(l)
      w_ij[l] = edge[-1]['weight']
    print("Finished Conversion")

    # Adding Variables to model
    protein_vars = model.addVars(ij, obj=w_ij, vtype=gp.GRB.BINARY)
    print("Finished Variable Creations")

    #Single Path Constraints
    for n in graph_nodes + ['s', 'd']: #adding s & d back just here
      v = str(n)
      model.addConstr(sum(protein_vars[i,j] for i,j in ij.select(v, '*')) - sum(protein_vars[j,i] for j,i in ij.select('*', v)) == (1 if v=='s' else -1 if v=='d' else 0), v)

    cnt=0
    # Intersection Constraints - forbidden primer pairs
    for cnt,pairs in enumerate(forbidden_pairs):
      all_edges = []
      for pair in pairs:
        node1 = (pair[0],pair[1],'f')
        node2 = (pair[0],pair[1],'r')
        all_edges.append(protein_vars.sum(str(node1), '*'))
        all_edges.append(protein_vars.sum(str(node2), '*'))
      model.addConstr(gp.quicksum(all_edges) <= 1)

    print("Finished single forbiden pairs constraints!")
    print("Number of single forbidden pairs: ",cnt)

    return protein_vars


## Add multiple protein constraints

In [ ]:
from itertools import combinations

def multiple_pairs(protein_names,sequences_nt):

    multiple_forbidden={}

    protein_pairs = list(combinations(range(len(protein_names)), 2))

    for p1,p2 in protein_pairs:
      sequence1=sequences_nt[p1]
      sequence2=sequences_nt[p2]

      protein1=protein_names[p1]
      protein2=protein_names[p2]

      pairs_finder = PairFinder(sequence1, sequence2)
      forbidden_pairs = pairs_finder.find_all_pairs()

      multiple_forbidden[(protein1,protein2)]=forbidden_pairs

    return multiple_forbidden


def multiple_constraints(model, model_vars, multiple_forbidden):

    total_forbidden_pairs = 0

    for proteins, forbidden_pairs in multiple_forbidden.items():

        protein1, protein2 = proteins

        x = model_vars[protein1]
        y = model_vars[protein2]

        for forbidden_pair in forbidden_pairs:
            # add <= constraint for ervey forbidden pair
            all_edges = []
            pair1, pair2 = forbidden_pair

            node_x1 = (pair1[0], pair1[1], 'f')
            node_x2 = (pair1[0], pair1[1], 'r')
            all_edges.append(x.sum(str(node_x1), '*'))
            all_edges.append(x.sum(str(node_x2), '*'))

            node_y1 = (pair2[0], pair2[1], 'f')
            node_y2 = (pair2[0], pair2[1], 'r')
            all_edges.append(y.sum(str(node_y1), '*'))
            all_edges.append(y.sum(str(node_y2), '*'))

            # constraint: the sum of outgoing edges of fobidden pairs <=1
            model.addConstr(gp.quicksum(all_edges) <= 1)
            total_forbidden_pairs += 1


    # Print the total number of constraints added after processing all combinations
    print("Finished adding multiple forbidden pairs constraints.")
    print("Total number of forbidden pairs constraints added:", total_forbidden_pairs)



## Forbidden pairs finder

In [ ]:
import multiprocessing
primer_lmin, primer_lmax = 18, 30
overlap = primer_lmax

class PairFinder:

  def __init__(self, sequence_nt1, sequence_nt2=None):
        self.sequence_nt1 = sequence_nt1
        self.sequence_nt2 = sequence_nt2 if sequence_nt2 else sequence_nt1

  def forbidden_pairs(self,primer, p_end, sequence_nt):

    memo={}

    def search(start, end):
        results = []

        # Base case
        if (end-start) <= 2*(overlap):

          tm = p3.bindings.calc_heterodimer(sequence_nt[start:end], primer, mv_conc=50.0, dv_conc=1.5, dntp_conc=0.6,
                                                  dna_conc=50.0, temp_c=37.0, max_loop=30).tm
          if tm>=max_tm:
            return handle_special_case(start, end)
          else:
            return []

        mid = (start + end) // 2
        left_end = mid + (overlap-1)
        right_start = max(mid - (overlap-1),0)

        first_tm = p3.bindings.calc_heterodimer(sequence_nt[start:left_end], primer, mv_conc=50.0, dv_conc=1.5, dntp_conc=0.6,
                                                  dna_conc=50.0, temp_c=37.0, max_loop=30).tm

        second_tm = p3.bindings.calc_heterodimer(sequence_nt[right_start:end], primer, mv_conc=50.0, dv_conc=1.5, dntp_conc=0.6,
                                            dna_conc=50.0, temp_c=37.0, max_loop=30).tm

        # Check and recurse on the left segment
        if first_tm >= max_tm:
            results.extend(search(start, left_end))

        # Check and recurse on the right segment
        if second_tm >= max_tm:
            results.extend(search(right_start, end))

        return results


    def handle_special_case(start, end):
        if (start, end) in memo:
            return memo[(start, end)]

        # Avoid unnecessary recursion if the segment is already at the minimum length
        if end - start <= primer_lmax:
            tm =  p3.bindings.calc_heterodimer(sequence_nt[start:end], primer, mv_conc=50.0, dv_conc=1.5, dntp_conc=0.6,
                                                  dna_conc=50.0, temp_c=37.0, max_loop=30).tm
            if tm >= max_tm:
                memo[(start, end)] = [(start, end)]
                return [(start, end)]
            else:
                memo[(start, end)] = []
                return []

        # Recursive case: try shortening the segment from both ends
        shorten_left = handle_special_case(start + 1, end)
        shorten_right = handle_special_case(start, end - 1)

        result = list(set(shorten_left+shorten_right))

        memo[(start, end)] = result
        return result

    off_targets=[]

    # search second half of sequence (with max allowed overlap)
    off_targets += search(max(p_end-allowed_overlap,0),len(sequence_nt))

    return off_targets

  def find_all_pairs(self):

    forbidden_pairs = []
    # iterate over every start position
    for p_start in range(len(self.sequence_nt1)-primer_lmax+1):
        p_end=p_start + primer_lmax
        primer = self.sequence_nt1[p_start:p_end]
        # single sequence logic
        if self.sequence_nt1 == self.sequence_nt2:
          off_targets = self.forbidden_pairs(primer,p_end,self.sequence_nt1)
        else: # multiple sequence logic
          off_targets = self.forbidden_pairs(primer,0,self.sequence_nt2)

        for target in off_targets:
            forbidden_pairs+=[((p_start,p_end),target)]

    forbidden_pairs=set(forbidden_pairs)

    sub_pairs = self.sub_pairs_parallel(forbidden_pairs)

    return sub_pairs

  def process_pair(self,pair):
    sub_forbidden_pairs = []
    p1, p2 = pair

    primer1 = self.sequence_nt1[p1[0]:p1[1]]
    primer2 = self.sequence_nt2[p2[0]:p2[1]]

    for length1 in range(primer_lmin, primer_lmax+1):
        for length2 in range(primer_lmin, primer_lmax+1):
            for start1 in range(primer_lmax - length1 + 1):
                for start2 in range(primer_lmax - length2 + 1):
                    sub1 = primer1[start1:start1 + length1]
                    sub2 = primer2[start2:start2 + length2]
                    tm = p3.bindings.calc_heterodimer(sub1, sub2, mv_conc=50.0, dv_conc=1.5, dntp_conc=0.6,
                                                      dna_conc=50.0, temp_c=37.0, max_loop=30).tm
                    if tm >= max_tm:
                        sub_forbidden_pairs.append(((start1+p1[0]-mutreg_start, start1 + p1[0] + length1-mutreg_start), (start2 + p2[0]-mutreg_start, start2 + p2[0] + length2 - mutreg_start)))

    return sub_forbidden_pairs

  def sub_pairs_parallel(self,forbidden_pairs):

      pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
      result = pool.map(self.process_pair, forbidden_pairs)
      pool.close()
      pool.join()

      # Flatten the list of lists
      sub_forbidden_pairs = set([item for sublist in result for item in sublist])

      return sub_forbidden_pairs


##Full Sequences
##### The user can choose the mutreg sequence and upstream and downstream regions

In [ ]:
# constant upstream and downstream regions for 10 proteins
upstream_nt = 'GCTAGTGGTGCTAGCCCCGCGAAATTAATACGACTCTCTATAGGGTCTAGAAATAATTTTCTTTAACTTTAAGAAGGAGATATACAT'
downstream_nt = 'GGAGGGTCTGGGGCAGGAGGCAGTGGCATGGTGACCAAGGGCGTGGAGCTGTTCACCGGGGTGGTGCCCATCCTGGTCGAGCTGGACGGCGACGTAAACGGCCACAAGTTCAGCGTGTCCGGCGAGGGCGAGGGCGATGCCACCTACGGCAAGCTGACCCTGAAGTTCATCTGCACCACCGGCAAGCTGCCCG'

mutreg_regions=[]
protein_names=[]

# read 10 protein coding sequences from file
with open('10 coding sequences.txt') as file:
  for line in file.readlines():
    p_name, mutreg_region = line.strip().split('\t')
    mutreg_regions.append(mutreg_region)
    protein_names.append(p_name)

sequences_nt=[]

for mutreg_nt in mutreg_regions:
  sequence= upstream_nt + mutreg_nt + downstream_nt
  sequences_nt.append(sequence)


##Parameters
##### The user can adjust algorithm parameters such as primer length, oligo length and allowed overlap

In [ ]:
# adjust algorithm parameters
primer_lmin, primer_lmax = 18, 30
overlap_lmin,overlap_lmax = 45,50
oligo_lmin,oligo_lmax = 195,205
allowed_overlap = 6
mutreg_start = len(upstream_nt)

# sets up pcr reaction with parameters
pcr = p3.thermoanalysis.ThermoAnalysis(dna_conc= 250,
                                       mv_conc= 50,
                                       dv_conc= 0,
                                       dntp_conc= 0,
                                       tm_method= 'santalucia',
                                       salt_correction_method= 'owczarzy',
                                       temp_c= 25)

## Efficiency Thresholds
##### The user can choose to apply primer efficiency threshold on gc content and tm and the maximum allowed tm different between forward and reverse primer in each pair

In [ ]:
apply_threshold= False # apply threshold flag. If true, threshold will be applied
min_gc=40
max_gc=60
min_tm=58
max_tm=65
max_difference=3

##Create primer graphs


In [ ]:
def create_graphs(mutreg_regions,sequences_nt,protein_names):

  #Creating the Graph
  start_time = time.time()
  tracemalloc.start()

  graphs={}
  primer_dfs={}

  for mutreg_nt,sequence,protein_name in zip(mutreg_regions,sequences_nt,protein_names):

    mutreg_l = len(mutreg_nt)

    primer_f, primer_df = create_primer_df(sequence)

    primer_dfs[protein_name] = primer_df

    G = nx.DiGraph()
    primers_init = [Primer(primer_df,p.start,p.stop) for _,p in primer_f.query('stop<=0').iterrows() if check_threshold(p.tm,p.gc)]  ## all forward primers that end before mutreg and pass threshold
    for primer in primers_init:
      G.add_edge('s',primer.tup(), weight=primer.w) #intializing the s-primer connection
      dfs(G, primer,primer_df,mutreg_l) #create the rest of the graph

    graphs[protein_name]=G

    print("Nodes: ", len(G.nodes))

  graph_time = int(time.time() - start_time)
  graph_memory = tracemalloc.get_traced_memory()[1] / 10**6 #MB
  tracemalloc.stop()

  all_nodes=0
  all_edges=0

  for G in graphs.values():
    all_nodes += len(G.nodes)
    all_edges += len(G.edges)

  return graphs, graph_time, graph_memory, all_nodes,all_edges, primer_dfs




## Model Creation and Run

##### This function creates the ILP solver and represents the primer graph as and ILP optimization problem. It adds the ILP contraints based on overlap and single path contraints. Then, it runs the ILP solver and returns primer path solution

In [ ]:

def run_ilp(single_forbidden,multiple_forbidden,protein_names,graphs):

  # measure setup time
  setup_start = time.time()
  tracemalloc.start()
  #Create ILP model
  model = get_model()

  # list of all groups of variables for each protein
  protein_vars={}

  ## call function to add single path and single forbidden pair ILP constraints
  for name in protein_names:
    graph = graphs[name]
    forbidden_pairs=single_forbidden[name]
    vars = single_constraints(graph,forbidden_pairs,model) # returns group of variables for protein
    protein_vars[name]=vars

  ## call function to add forbidden pair constraint between each protein pair
  multiple_constraints(model, protein_vars, multiple_forbidden)

  setup_time = time.time() - setup_start
  setup_memory = tracemalloc.get_traced_memory()[1] / 10**6
  tracemalloc.stop()

  tracemalloc.start()
  start_time = time.time()
  model.optimize()
  ILP_time = time.time() - start_time
  ILP_memory = tracemalloc.get_traced_memory()[1] / 10**6
  tracemalloc.stop()

  def post_processing(x):
    path = ['s']
    true_edges = [index for index, var in x.items() if var.X != 0]

    while true_edges:
      edge = true_edges.pop(0)
      added_edge = False
      if edge[0] == path[-1]:
        path.append(edge[1])
        added_edge = True
      if not added_edge:
        true_edges.append(edge)

    return path

  protein_paths={}

  for name,x in protein_vars.items():
    actual_values = post_processing(x)
    protein_paths[name]=actual_values
  for name,vals in protein_paths.items():
    print(f"Protein #{name} ({len(vals)})")
    print(vals)
    print()

  return model.numVars, model.numConstrs, setup_time, setup_memory, ILP_time, ILP_memory, protein_paths, model.objVal

## Run ILP and greedy algorithm


In [ ]:
all_data = []
max_tm=45

# iterate over all protein numbers from 2 to 10
for i in range(2,11):

  ## create graphs
  graphs, graph_time, graph_memory,all_nodes, all_edges, primer_dfs = create_graphs(mutreg_regions[:i],sequences_nt[:i],protein_names[:i])

  start_time = time.time()

  # call function create dictionary of forbidden primer pairs per single protein
  single_forbidden = single_pairs(protein_names[:i],sequences_nt[:i])

  single_pairs_time=time.time()-start_time

  start_time = time.time()

  # call function that returns forbidden protein pair between multiple proteins
  multiple_forbidden = multiple_pairs(protein_names[:i],sequences_nt[:i])

  multi_pairs_time=time.time()-start_time

  ## Greedy Solution
  start_time = time.time()
  greedy_solution, greedy_obj = run_greedy(graphs,primer_dfs,multiple_forbidden,protein_names[:i])
  greedy_time = time.time() - start_time

  # call funstion to run ILP
  numVars, numConstrs, setup_time, setup_memory, ILP_time, ILP_memory, protein_paths, objective = run_ilp(single_forbidden,multiple_forbidden,protein_names[:i],graphs)


  all_data.append({"num proteins:": len(protein_names),
                    "Nodes":all_nodes,
                    "Edges": all_edges,
                    "Time (Graph)": graph_time,
                    "MP (Graph)": graph_memory,
                    "Vars": numVars,
                    "Constr": numConstrs,
                    "Time (Setup)": setup_time,
                    "MP (Setup)": setup_memory,
                    "multi pair time": multi_pairs_time,
                    "single pair time": single_pairs_time,
                    "Time (ILP)": ILP_time,
                    "MP (ILP)": ILP_memory,
                    "ILP Solution": protein_paths,
                    "ILP Objective": objective,
                    "Greedy Solution": greedy_solution,
                    "Greedy Objective": greedy_obj,
                    "Greedy Time": greedy_time})

  print(all_data[-1])

  run_df = pd.DataFrame(all_data)

  # Write the DataFrame to a CSV file
  run_df.to_csv('run_data.csv', index=False)



/usr/local/lib/python3.10/dist-packages/pandas/core/algorithms.py:1743: UserWarning: Function deprecated please use "calc_tm" instead
  return lib.map_infer(values, mapper, convert=convert)
<ipython-input-4-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin" instead
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
<ipython-input-4-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin" instead
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
<ipython-input-4-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin" instead
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
<ipython-input-4-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin" instead
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
<ipython-input-4-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin" inst

Nodes:  25010
0
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2458238


GurobiError: License has expired